In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.randn(2, 20)
print(net(X))

tensor([[-0.4590,  0.5837,  0.3562,  0.0610, -0.3602,  0.3285, -0.2436, -0.6119,
         -0.3299, -0.3520],
        [ 0.0305,  0.1996, -0.1504,  0.0093, -0.0923,  0.1383, -0.0782, -0.1004,
         -0.0944, -0.2263]], grad_fn=<AddmmBackward0>)


In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
print(net(X))

tensor([[ 0.2747, -0.1260,  0.3884, -0.4801, -0.2002, -0.0230,  0.1368,  0.0729,
         -0.6116,  0.0993],
        [ 0.1298,  0.2135, -0.1579, -0.1591, -0.1208, -0.0960,  0.1958,  0.2448,
         -0.3187, -0.1000]], grad_fn=<AddmmBackward0>)


In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
print(net(X))

tensor([[ 0.1238,  0.2121, -0.1678,  0.0442, -0.3382, -0.7192,  0.2220, -0.2084,
         -0.2544, -0.3818],
        [ 0.3688, -0.0765, -0.0197, -0.1659, -0.1894,  0.0914,  0.1204, -0.0910,
          0.0658, -0.1833]], grad_fn=<AddmmBackward0>)


In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.matmul(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
print(net(X))

tensor(-0.0111, grad_fn=<SumBackward0>)


In [6]:
class NestedMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.out = nn.Linear(32, 16)

    def forward(self, X):
        return self.out(self.net(X))
    
chimera = nn.Sequential(
    NestedMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
print(chimera(X))

tensor(0.3952, grad_fn=<SumBackward0>)
